In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [104]:
class GridWorld:
    
    def __init__(self, n, terminal_state, inplace = False, policy_iteration = True, verbose=False):
        
        self.grid_size = n
        self.terminal_state = terminal_state
        self.inplace = inplace
        self.policy_iteration = policy_iteration
        self.verbose = verbose
        self.gamma = 0.1
        
        self.vt = np.zeros((self.grid_size, self.grid_size))
        if not self.inplace:
            self.vt1 = np.zeros((self.grid_size, self.grid_size))
        
        self.actions = [(0,-1), (0, 1), (-1,0), (1, 0)]# Left, Right, Up, Down

        #Equi-probable action selection at each state for all actions
        self.policy = np.random.randint(1,4,size = (self.grid_size, self.grid_size))
        
        print("Initial V", *self.vt, sep='\n',end='\n\n')
        print("Initial Policy", *self.policy, sep='\n',end='\n\n')
        
        if self.verbose:
            print("Shape of V", self.vt.shape)
            print("Shape of Policy", self.policy.shape)
        
    def _get_reward(self, x, y):
        if x==self.terminal_state[0] and y==self.terminal_state[1]:
            return 3
        return -1
    
    def _get_next_state(self, x, y, a):
        
        if x+a[0]!=-1 and x+a[0]!=self.grid_size:
            x = x + a[0]
        if y+a[1]!=-1 and y+a[1]!=self.grid_size:
            y = y + a[1]
            
        return x,y
        
    def play(self, epochs = 10, threshold = 0.1):
        
        if self.policy_iteration:
            res = self._play_policy_iteration(epochs, threshold)
            print("Final Policy:")
            print(*self.policy, sep = '\n', end = '\n\n')
            print("Final Value Function:")
            print(*self.vt, sep = '\n', end = '\n\n')
            return res
        
        else:
            res = self._play_value_iteration(epochs, threshold)
            print("Final Policy:")
            print(*self.policy, sep = '\n', end = '\n\n')
            print("Final Value Function:")
            print(*self.vt, sep = '\n', end = '\n\n')
            return res
        
    def _play_policy_iteration(self, epochs, threshold):
        
        
        iters = 0
        while True:
            
            iters+=1
            
            #Policy Evaluation:
#             for _ in range(iters):
            diff = 0
            for i in range(self.grid_size):
                for j in range(self.grid_size):

                    old_V = self.vt[i, j]
                    action = self.actions[self.policy[i,j]]
                    new_x, new_y = self._get_next_state(i,j,action)
                    reward = self._get_reward(new_x, new_y)

                    if self.inplace:
                        self.vt[i,j] = 0.25*(reward + self.gamma * self.vt[new_x, new_y])
                        diff+= abs(old_V-self.vt[i,j])
                    else:
                        self.vt1[i,j] = 0.25*(reward + self.gamma * self.vt[new_x, new_y])
                        diff+= abs(old_V-self.vt1[i,j])

            if not self.inplace:
                self.vt = np.array(self.vt1)
                
#                 if diff <= threshold:
#                     break
                    
            if self.verbose and not self.inplace:
                print("Ids of Vt and Vt+1", id(self.vt), id(self.vt1))
            
            if self.verbose:
                print("Value function at iter:", iters)
                print(*self.vt, sep = '\n', end = '\n\n')
            
            #Policy Improvement:
            for i in range(self.grid_size):
                for j in range(self.grid_size):
                    
                    best_action = -1
                    best_value = -10
                    for k, a in enumerate(self.actions):
                        
                        new_x, new_y = self._get_next_state(i,j,a)
                        reward = self._get_reward(new_x, new_y)
                        if best_value< (0.25*(reward + self.gamma*self.vt[new_x, new_y])):
                            best_value = (0.25*(reward + self.gamma*self.vt[new_x, new_y]))
                            best_action = k
                    
                    self.policy[i,j] = best_action
            
            if self.verbose:
                print("Policy at iter:", iters)
                print(*self.policy, sep = '\n', end = '\n\n')
            
            if epochs==iters or diff<=threshold:
                return diff, iters

    def _play_value_iteration(self, epochs, threshold):
        
        
        iters = 0
        while True:
            
            diff = 0
            for i in range(self.grid_size):
                for j in range(self.grid_size):
                    
                    old_V = self.vt[i, j]
                    best_action = -1
                    best_value = -10
                    for k, a in enumerate(self.actions):
                        
                        new_x, new_y = self._get_next_state(i,j,a)
                        reward = self._get_reward(new_x, new_y)
                        if best_value< (0.25*(reward + self.gamma*self.vt[new_x, new_y])):
                            best_value = (0.25*(reward + self.gamma*self.vt[new_x, new_y]))
                            best_action = k
                            
                    if self.inplace:
                        self.vt[i,j] = best_value
                        diff+= abs(old_V-self.vt[i,j])
                    else:
                        self.vt1[i,j] = best_value
                        diff+= abs(old_V-self.vt1[i,j])
                        
                    self.policy[i,j] = best_action
            
            iters+=1
            
            if not self.inplace:
                self.vt = np.array(self.vt1)
            
            if self.verbose and not self.inplace:    
                print("Ids of Vt and Vt+1", id(self.vt), id(self.vt1))
            
            if self.verbose:
                print("Value function at iter:", iters)
                print(*self.vt, sep = '\n', end = '\n\n')
            
            if self.verbose:
                print("Policy at iter:", iters)
                print(*self.policy, sep = '\n', end = '\n\n')
            
            if epochs==iters or diff<=threshold:
                return diff, iters


# Policy Iteration

In [105]:
#Inplace
game = GridWorld(4, (2,1), inplace=True, policy_iteration = True)
print(game.play(epochs=50, threshold=0.00000000000001))

Initial V
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]

Initial Policy
[2 3 1 3]
[2 2 2 3]
[2 3 3 2]
[1 2 1 1]

Final Policy:
[1 3 0 0]
[1 3 0 0]
[1 0 0 0]
[1 2 0 0]

Final Value Function:
[-0.25578487 -0.23139462 -0.25578487 -0.25639462]
[-0.23139462  0.74421513 -0.23139462 -0.25578487]
[ 0.74421513 -0.23139462  0.74421513 -0.23139462]
[-0.23139462  0.74421513 -0.23139462 -0.25578487]

(1.1102230246251565e-16, 8)


In [106]:
game = GridWorld(4, (2,1), inplace=False, policy_iteration = True)
print(game.play(epochs=50, threshold=0.00000000000001))

Initial V
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]

Initial Policy
[3 2 1 1]
[1 2 2 2]
[3 3 3 3]
[3 3 1 1]

Final Policy:
[1 3 0 0]
[1 3 0 0]
[1 0 0 0]
[1 2 0 0]

Final Value Function:
[-0.25578487 -0.23139462 -0.25578487 -0.25639462]
[-0.23139462  0.74421513 -0.23139462 -0.25578487]
[ 0.74421513 -0.23139462  0.74421513 -0.23139462]
[-0.23139462  0.74421513 -0.23139462 -0.25578487]

(8.881784197001252e-16, 12)


# Value Iteration

In [107]:
#Inplace
game = GridWorld(4, (2,1), inplace=True, policy_iteration = False)
print(game.play(epochs=50, threshold=0.00000000000001))

Initial V
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]

Initial Policy
[1 3 2 2]
[3 2 1 2]
[2 1 3 3]
[1 2 3 2]

Final Policy:
[1 3 0 0]
[1 3 0 0]
[1 0 0 0]
[1 2 0 0]

Final Value Function:
[-0.25578487 -0.23139462 -0.25578487 -0.25639462]
[-0.23139462  0.74421513 -0.23139462 -0.25578487]
[ 0.74421513 -0.23139462  0.74421513 -0.23139462]
[-0.23139462  0.74421513 -0.23139462 -0.25578487]

(8.881784197001252e-16, 7)


In [109]:
game = GridWorld(4, (2,1), inplace=False, policy_iteration = False)
print(game.play(epochs=50, threshold=0.00000000000001))

Initial V
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]

Initial Policy
[2 2 2 3]
[1 2 3 2]
[3 1 2 2]
[3 2 2 3]

Final Policy:
[1 3 0 0]
[1 3 0 0]
[1 0 0 0]
[1 2 0 0]

Final Value Function:
[-0.25578487 -0.23139462 -0.25578487 -0.25639462]
[-0.23139462  0.74421513 -0.23139462 -0.25578487]
[ 0.74421513 -0.23139462  0.74421513 -0.23139462]
[-0.23139462  0.74421513 -0.23139462 -0.25578487]

(6.38378239159465e-16, 11)


## The results clearly shows that inplace updation along with the Value Iteration converges fastest.